# Manpower Optimisation Labs
by Matara Gerald

## Hungarian Assignment Algorithm

The Hungarian Assignment Algorithm, also known as the Kuhn-Munkres algorithm, is a combinatorial optimization algorithm that solves the assignment problem in polynomial time. The assignment problem involves finding the optimal way to assign a set of workers to a set of tasks such that the total cost is minimized (or maximized, depending on the problem).

### Key Features:
- **Input**: A cost matrix where each element represents the cost of assigning a specific worker to a specific task.
- **Output**: An optimal assignment of workers to tasks that minimizes the total cost.
- **Applicability**: Works for both square (n x n) and rectangular (n x m) matrices. In rectangular cases, some workers or tasks may remain unassigned.

### Steps of the Algorithm:
1. Subtract the smallest value in each row from all elements of that row.
2. Subtract the smallest value in each column from all elements of that column.
3. Cover all zeros in the matrix using the minimum number of horizontal and vertical lines.
4. If the number of lines equals the size of the matrix, an optimal assignment is found. Otherwise, adjust the matrix and repeat.

### Advantages:
- Efficient for solving large-scale assignment problems.
- Guarantees an optimal solution.

### Applications:
- Task scheduling.
- Resource allocation.
- Matching problems in graph theory.

In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment
import sys

def solve_hungarian_assignment(cost_matrix):
    """
    Solves the assignment problem (Hungarian Algorithm) to find the
    minimum cost matching between workers and tasks.

    Args:
        cost_matrix (np.ndarray or list of lists): A 2D array where
            cost_matrix[i, j] is the cost of assigning worker i to task j.
            - If square (n x n): Finds the perfect matching minimizing cost.
            - If rectangular (n x m): Finds the best assignment of min(n, m)
              pairs. If n > m (more workers), some workers are unassigned.
              If m > n (more tasks), some tasks are unassigned.

    Returns:
        tuple: A tuple containing:
            - list: A list of tuples representing the optimal assignments,
                    e.g., [(worker_index, task_index), ...].
            - float: The total minimum cost of the optimal assignment.
            - list: Indices of workers included in the assignment.
            - list: Indices of tasks included in the assignment.

    Raises:
        ValueError: If the cost matrix is not valid (e.g., not 2D).
    """
    cost_matrix = np.asarray(cost_matrix) # Ensure it's a NumPy array

    if cost_matrix.ndim != 2:
        raise ValueError("Cost matrix must be 2-dimensional.")

    if cost_matrix.size == 0:
        print("Warning: Empty cost matrix provided.")
        return [], 0.0, [], []

    num_workers, num_tasks = cost_matrix.shape
    print(f"Input Cost Matrix ({num_workers} workers x {num_tasks} tasks):\n{cost_matrix}\n")

    # Use scipy's linear_sum_assignment which implements the Hungarian algorithm
    # It returns optimal row indices and corresponding column indices.
    try:
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
    except ValueError as e:
        # Catch potential issues like non-finite values if not handled prior
        print(f"Error during assignment: {e}", file=sys.stderr)
        print("Ensure cost matrix contains only finite numbers.", file=sys.stderr)
        raise # Re-raise the error after printing info


    # Extract the assignments and calculate the total cost
    assignments = list(zip(row_ind, col_ind))
    total_cost = cost_matrix[row_ind, col_ind].sum()

    print("Optimal Assignments (Worker Index -> Task Index):")
    assigned_workers = set()
    assigned_tasks = set()
    for r, c in assignments:
        print(f"  Worker {r} -> Task {c} (Cost: {cost_matrix[r, c]})")
        assigned_workers.add(r)
        assigned_tasks.add(c)

    print(f"\nTotal Minimum Cost: {total_cost}")

    # Identify unassigned if rectangular
    if num_workers > num_tasks:
        unassigned_workers = set(range(num_workers)) - assigned_workers
        if unassigned_workers:
            print(f"Unassigned Workers: {sorted(list(unassigned_workers))}")
    elif num_tasks > num_workers:
        unassigned_tasks = set(range(num_tasks)) - assigned_tasks
        if unassigned_tasks:
            print(f"Unassigned Tasks: {sorted(list(unassigned_tasks))}")

    return assignments, total_cost, sorted(list(assigned_workers)), sorted(list(assigned_tasks))

### Example Usage: Minimization (Square Matrix)

In [ ]:
cost_matrix_min = np.array([
    [90, 76, 75],
    [35, 85, 55],
    [125, 95, 90]
])
assignments_min, cost_min, _, _ = solve_hungarian_assignment(cost_matrix_min)


Input Cost Matrix (3 workers x 3 tasks):
[[ 90  76  75]
 [ 35  85  55]
 [125  95  90]]

Optimal Assignments (Worker Index -> Task Index):
  Worker 0 -> Task 1 (Cost: 76)
  Worker 1 -> Task 0 (Cost: 35)
  Worker 2 -> Task 2 (Cost: 90)

Total Minimum Cost: 201


### Example Usage: Rectangular Matrix (More Workers than Tasks)

In [ ]:
cost_matrix_rect = np.array([
    [90, 76],
    [35, 85],
    [125, 95]
])

assignments_rect, cost_rect, assigned_w, assigned_t = solve_hungarian_assignment(cost_matrix_rect)

Input Cost Matrix (3 workers x 2 tasks):
[[ 90  76]
 [ 35  85]
 [125  95]]

Optimal Assignments (Worker Index -> Task Index):
  Worker 0 -> Task 1 (Cost: 76)
  Worker 1 -> Task 0 (Cost: 35)

Total Minimum Cost: 111
Unassigned Workers: [2]
